In [16]:
%pip install urllib3


[notice] A new release of pip available: 22.2.2 -> 24.2
[notice] To update, run: pip3.10 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [17]:
%pip install requests


[notice] A new release of pip available: 22.2.2 -> 24.2
[notice] To update, run: pip3.10 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [18]:
%pip install lxml


[notice] A new release of pip available: 22.2.2 -> 24.2
[notice] To update, run: pip3.10 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [19]:
%pip install pandas


[notice] A new release of pip available: 22.2.2 -> 24.2
[notice] To update, run: pip3.10 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [20]:
%pip install selenium


[notice] A new release of pip available: 22.2.2 -> 24.2
[notice] To update, run: pip3.10 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [36]:
import ssl
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import json

context = ssl._create_unverified_context() 
base_url = "https://www.worldmarket.com/c/new-and-trending/"
page_num = 1
max_pages = 13 #halaman

nama_produk = []
harga_produk = []
pick_up_info = []
arrive_info = []

while page_num <= max_pages:
    url = f"{base_url}?page={page_num}"
    html = urlopen(url, context=context).read()
    
    soup = BeautifulSoup(html, 'html.parser')
    
    produk = soup.find_all("div", "col-6 col-lg-4 product-tile-wrapper product-tile-wrapper-even js-a-tile-data js-lazy-tile")
    
    for p in produk:
        product_name = p.find('a', 'link js-a-product-click').get_text().strip()
        value_span = p.find('span', 'value')
        product_value = ''.join([text for text in value_span.contents if not text.name]).replace('$','').replace('.', '').strip()
        
        pick_up_div = p.find('div', class_='tab tab-pick-up')
        if pick_up_div:
            pick_up_text = pick_up_div.find('span', class_='text text-sm text-color-t200 medium').get_text(strip=True)
            pick_up_days = pick_up_div.find('b').get_text(strip=True)
            pick_up = f"{pick_up_text} {pick_up_days}"
        else:
            pick_up = "N/A"

        arrive_div = p.find('div', class_='tab tab-shipping')
        if arrive_div:
            arrive_text = arrive_div.find('span', class_='text text-sm text-color-t200 medium').get_text(strip=True)
            arrive = arrive_text
        else:
            arrive = "N/A"

        # Append the data to lists
        nama_produk.append(product_name)
        harga_produk.append(product_value)
        pick_up_info.append(pick_up)
        arrive_info.append(arrive)

    page_num += 1  # Move to the next page

# Create a DataFrame and sort by 'Harga'
product_dict = {
    'Nama': nama_produk,
    'Harga': harga_produk,
    'Pick Up': pick_up_info,
    'Arrive': arrive_info
}

df = pd.DataFrame(product_dict, columns=['Nama', 'Harga', 'Pick Up', 'Arrive'])
df['Harga'] = df['Harga'].astype(float)  # Convert prices to float for proper sorting
df_sorted = df.sort_values('Harga', ascending=True)

# Save to CSV
df_sorted.to_csv("WorldMarket.csv", sep=',', index=False)

# Save to JSON
products = [{'Nama': nama_produk[i], 'Harga': harga_produk[i]} for i in range(len(nama_produk))]
shipping_info = [{'Nama': nama_produk[i], 'Pick Up': pick_up_info[i], 'Arrive': arrive_info[i]} for i in range(len(nama_produk))]

with open('data/products.json', 'w', encoding='utf-8') as f:
    json.dump(products, f, ensure_ascii=False, indent=4)

with open('data/shipping_info.json', 'w', encoding='utf-8') as f:
    json.dump(shipping_info, f, ensure_ascii=False, indent=4)


In [23]:
import ssl
from urllib.request import urlopen

context = ssl._create_unverified_context()  #matiin SSL verif

url = "https://www.worldmarket.com/c/new-and-trending/"
html = urlopen(url, context=context).read()
print(type(html))   #print tipe html
print(html[1:1000])  #1000 karakter pertama (source code)

<class 'bytes'>
b'\n\n\n\n\n\n<!DOCTYPE html>\n<html lang="en">\n<head>\n<link rel="canonical" href="https://www.worldmarket.com/c/new-and-trending/" />\n<link rel="next" href="https://www.worldmarket.com/c/new-and-trending/?start=59&amp;sz=59" />\n<!--[if gt IE 9]><!-->\n<script>//common/scripts.isml</script>\n<script defer type="text/javascript" src="/on/demandware.static/Sites-World_Market-Site/-/en_US/v1722582241177/js/main.js"></script>\n<script defer type="text/javascript" src="/on/demandware.static/Sites-World_Market-Site/-/en_US/v1722582241177/js/search.js">\n        </script>\n<script defer type="text/javascript" src="/on/demandware.static/Sites-World_Market-Site/-/en_US/v1722582241177/js/aCategoryTrigger.js">\n        </script>\n<!--<![endif]-->\n<meta charset="UTF-8">\n<meta http-equiv="x-ua-compatible" content="ie=edge">\n<meta name="viewport" content="width=device-width, initial-scale=1">\n<title>New &amp; Trending Home Decor &amp; Furniture - World Market</title>\n<meta n

In [24]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(html, "html.parser")
print(type(soup))
print(soup.prettify()[1:1000])

<class 'bs4.BeautifulSoup'>
!DOCTYPE html>
<html lang="en">
 <head>
  <link href="https://www.worldmarket.com/c/new-and-trending/" rel="canonical"/>
  <link href="https://www.worldmarket.com/c/new-and-trending/?start=59&amp;sz=59" rel="next"/>
  <!--[if gt IE 9]><!-->
  <script>
   //common/scripts.isml
  </script>
  <script defer="" src="/on/demandware.static/Sites-World_Market-Site/-/en_US/v1722582241177/js/main.js" type="text/javascript">
  </script>
  <script defer="" src="/on/demandware.static/Sites-World_Market-Site/-/en_US/v1722582241177/js/search.js" type="text/javascript">
  </script>
  <script defer="" src="/on/demandware.static/Sites-World_Market-Site/-/en_US/v1722582241177/js/aCategoryTrigger.js" type="text/javascript">
  </script>
  <!--<![endif]-->
  <meta charset="utf-8"/>
  <meta content="ie=edge" http-equiv="x-ua-compatible"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <title>
   New &amp; Trending Home Decor &amp; Furniture - World Marke

In [25]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(html, 'html.parser')

produk = soup.find_all("div", "col-6 col-lg-4 product-tile-wrapper product-tile-wrapper-even js-a-tile-data js-lazy-tile")

for i, p in enumerate(produk, start=1):
    product_name = p.find('a', 'link js-a-product-click').get_text().strip()
    value_span = p.find('span', 'value')
    pick_up = p.find('span', 'text text-sm text-color-t200 medium').get_text()
    arrive = p.find('span', 'text text-sm text-color-t200 medium').get_text()
    product_value = ''.join([text for text in value_span.contents if not text.name]).replace('$','').replace('.','')
    print(product_name, product_value, pick_up, arrive)


Black Multicolor Mushroom Forest Coir Doormat 

1699

 
Pick Up Today
 
Pick Up Today

Coir Rope Knot Doormat 

2999

 
Pick Up Today
 
Pick Up Today

Red Cherry Shaped Indoor Outdoor Throw Pillow 

2999

 
Pick Up Today
 
Pick Up Today

Olive Green Embroidered Cranes Throw Pillow 

2999

 
Pick Up Today
 
Pick Up Today

This Is Fine Talking Figurine Mini Kit 

1395

 
Pick Up Today
 
Pick Up Today

Dutton Round Marble Top and Gold Metal Side Table 

7999

 
Pick Up in 7-14 Days
 
Pick Up in 7-14 Days

Green Avocado Shaped Tufted Indoor Outdoor Throw Pillow 

2999

 
Pick Up Today
 
Pick Up Today

Natural Green Marble Fluted Trinket Box 

2499

 
Pick Up Today
 
Pick Up Today

Kenwick Round Marble Top and Wood Nesting Tables 2 Piece Set 

39999

 
Pick Up in 7-14 Days
 
Pick Up in 7-14 Days

Keogh's Truffle and Irish Butter Potato Chips Snack Size 

149

 
Pick Up Today
 
Pick Up Today

Antique Gold Metal Orb Taper Candle Holder 

1299

 
Pick Up Today
 
Pick Up Today

Copper Patina Me

In [26]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(html, 'html.parser')

nama_produk = []
harga_produk = []
pick_up_info = []
arrive_info = []

produk = soup.find_all("div", "col-6 col-lg-4 product-tile-wrapper product-tile-wrapper-even js-a-tile-data js-lazy-tile")

for i, p in enumerate(produk, start=1):
    product_name = p.find('a', 'link js-a-product-click').get_text().strip()
    value_span = p.find('span', 'value')
    product_value = ''.join([text for text in value_span.contents if not text.name]).replace('$','').replace('.', '').strip()

    pick_up_div = p.find('div', class_='tab tab-pick-up')
    if pick_up_div:
        pick_up_text = pick_up_div.find('span', class_='text text-sm text-color-t200 medium').get_text(strip=True)
        pick_up_days = pick_up_div.find('b').get_text(strip=True)
        pick_up = f"{pick_up_text} {pick_up_days}"
    else:
        pick_up = "N/A"

    arrive_div = p.find('div', class_='tab tab-shipping')
    if arrive_div:
        arrive_text = arrive_div.find('span', class_='text text-sm text-color-t200 medium').get_text(strip=True)
        arrive = arrive_text
    else:
        arrive = "N/A"

    nama_produk.append(product_name)
    harga_produk.append(product_value)
    pick_up_info.append(pick_up)
    arrive_info.append(arrive)

    print(product_name, product_value, pick_up, arrive)

Black Multicolor Mushroom Forest Coir Doormat 1699 Pick UpToday Today Arrives in 4-10 Days
Coir Rope Knot Doormat 2999 Pick UpToday Today Arrives in 4-10 Days
Red Cherry Shaped Indoor Outdoor Throw Pillow 2999 Pick UpToday Today Arrives in 4-10 Days
Olive Green Embroidered Cranes Throw Pillow 2999 Pick UpToday Today Arrives in 4-10 Days
This Is Fine Talking Figurine Mini Kit 1395 Pick UpToday Today Arrives in 4-10 Days
Dutton Round Marble Top and Gold Metal Side Table 7999 Pick Up in7-14 Days 7-14 Days Arrives in 9-19 Days
Green Avocado Shaped Tufted Indoor Outdoor Throw Pillow 2999 Pick UpToday Today Arrives in 4-10 Days
Natural Green Marble Fluted Trinket Box 2499 Pick UpToday Today Arrives in 4-10 Days
Kenwick Round Marble Top and Wood Nesting Tables 2 Piece Set 39999 Pick Up in7-14 Days 7-14 Days Arrives in 9-19 Days
Keogh's Truffle and Irish Butter Potato Chips Snack Size 149 Pick UpToday Today N/A
Antique Gold Metal Orb Taper Candle Holder 1299 Pick UpToday Today Arrives in 4-10 

Panda Dataframe


In [33]:
import pandas as pd

product_dict = {
    'Nama': nama_produk,
    'Harga': harga_produk,
    'Pick Up': pick_up_info,
    'Arrive': arrive_info
}

df = pd.DataFrame(product_dict, columns=['Nama', 'Harga', 'Pick Up', 'Arrive'])
df['Harga'] = df['Harga'].astype(float) 
df_sorted = df.sort_values('Harga', ascending=True)

print(df_sorted)


                                                 Nama    Harga  \
9   Keogh's Truffle and Irish Butter Potato Chips ...    149.0   
20           Gold Modern Cocktail Flatware Collection    299.0   
24   Bonne Maman Chocolate Shortbread Cookies 14 Pack    749.0   
12                                 Twig Serving Spoon    799.0   
23                  Green Boucle Fabric Pumpkin Decor    799.0   
25                  Natural Rubber Wood Slotted Spoon    799.0   
15  The Republic of Tea Tangerine Yuzu Cold Brew T...   1199.0   
17       Cafe Mexicano Veracruz Reserve Ground Coffee   1199.0   
10         Antique Gold Metal Orb Taper Candle Holder   1299.0   
4              This Is Fine Talking Figurine Mini Kit   1395.0   
0       Black Multicolor Mushroom Forest Coir Doormat   1699.0   
14                       Fall Fragrance Reed Diffuser   1799.0   
7             Natural Green Marble Fluted Trinket Box   2499.0   
3         Olive Green Embroidered Cranes Throw Pillow   2999.0   
2       Re

In [34]:
df.to_csv("WorldMarket.csv", sep=',')

In [35]:
import json
products = [{'Nama': nama_produk[i], 'Harga': harga_produk[i]} for i in range(len(nama_produk))]
shipping_info = [{'Nama': nama_produk[i], 'Pick Up': pick_up_info[i], 'Arrive': arrive_info[i]} for i in range(len(nama_produk))]

with open('data/products.json', 'w', encoding='utf-8') as f:
    json.dump(products, f, ensure_ascii=False, indent=4)

with open('data/shipping_info.json', 'w', encoding='utf-8') as f:
    json.dump(shipping_info, f, ensure_ascii=False, indent=4)